In [9]:
%load_ext autoreload
%autoreload 2

import datetime
import multiprocessing
import multiprocess
import feedparser as fp
import newspaper
from newspaper import news_pool
import helpers
from newspaper import Source, Article
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%%time
# Do conventional newspaper3k build...
cnn_3k_paper = newspaper.build('https://www.cnn.com/', memoize_articles=False)
rt_3k_paper = newspaper.build('https://www.rt.com/', memoize_articles=False)
bbc_3k_paper = newspaper.build('https://www.bbc.com/news', memoize_articles=False)
fox_3k_paper = newspaper.build('https://www.foxnews.com/', memoize_articles=False)
msnbc_3k_paper = newspaper.build('https://www.msnbc.com/', memoize_articles=False)

CPU times: user 9.97 s, sys: 1.51 s, total: 11.5 s
Wall time: 11.2 s


In [11]:
%%time
# ... and multithread download:
papers = [cnn_3k_paper, 
          rt_3k_paper,
          bbc_3k_paper,
          fox_3k_paper,
          msnbc_3k_paper]
news_pool.set(papers, threads_per_source=2) # (5*2) = 10 threads total
news_pool.join()

CPU times: user 1min 10s, sys: 10.2 s, total: 1min 20s
Wall time: 6min 10s


In [12]:
%%time
# Process the downloaded texts:
for paper in papers:
    paper.parse_articles()

CPU times: user 7min 23s, sys: 18.3 s, total: 7min 41s
Wall time: 8min 4s


In [18]:
filepath = 'data/conventional_data_pulls/'

In [19]:
%%time
for paper in papers:
    helpers.write_out_articles(newspaper_obj=paper,
                               filepath=filepath, 
                               writeout_type='conventional')

CPU times: user 27 s, sys: 4.36 s, total: 31.4 s
Wall time: 32.1 s


In [112]:
df = pd.read_csv('data/conventional_data_pulls/2019_1_31_bbc.csv')

In [115]:
df['text'][95]

'Image copyright Reuters Image caption The Federal Reserve has been gradually raising interest rates since 2015\n\nThe Federal Reserve has indicated it won\'t raise interest rates anytime soon, marking an abrupt turnaround from earlier statements that suggested it would gradually increase rates.\n\nThe US central bank pledged to be "patient" citing muted inflation and recent economic "cross currents".\n\nIt made its statement after the Fed\'s January meeting, where it held interest rates unchanged, as expected.\n\nMarkets, which have been worried about higher rates, jumped on the news.\n\nThe Dow Jones Industrial Average ended the day more than 1.75% higher, the S&P 500 gained 1.56% and the Nasdaq increased 2.2%.\n\n\'Uncertainties\'\n\nThe Fed has been slowly raising rates since 2015, arguing that a stronger US economy meant the ultra-low rates put in place during the financial crisis were no longer necessary.\n\nBut Federal Reserve Chair Jerome "Jay" Powell said changes in recent mon